In [ ]:
import pandas as pd
import glob
from fastai.vision import *
import os

In [ ]:
path = '../input/severstal-steel-defect-detection/'

In [ ]:
path_train = '../input/severstal-steel-defect-detection/train_images/'

In [ ]:
path_test = '../input/severstal-steel-defect-detection/test_images/'

In [ ]:
df = pd.read_csv(path + 'train.csv')

In [ ]:
df = df[:100]

In [ ]:
df.head()

In [ ]:
hist = df['ClassId'].hist(bins=4)

In [ ]:
df_2 = df.groupby(['ImageId']).agg({'ClassId': lambda x: list(x),'EncodedPixels': lambda x: list(x)})

In [ ]:
df_2.head()

In [ ]:
df_2['ImageId'] = df_2.index

In [ ]:
df_2.head()

In [ ]:
df_2 = df_2.reset_index(drop=True)

In [ ]:
df_2.head()

In [ ]:
df.columns.tolist()

In [ ]:
df_2 = df_2[['ImageId', 'ClassId', 'EncodedPixels']]

In [ ]:
df_2.head()

In [ ]:
def func(x, y):
    a = ['','','','']
    for i, j in zip(x, y):
        a[i-1] = j
        
    return a      


In [ ]:
df_2['EncodedPixels'] = df_2.apply(lambda x: func(x['ClassId'], x['EncodedPixels']), axis=1)

In [ ]:
df_2.head(10)

In [ ]:
img = open_image(path_train + df_2['ImageId'][5])
img.show(figsize=(20,10))
print(img.size)

Маски для тренировочной выборки(train_images) закодированы(Run Lenght Encode или RLE) - "кодирование длины пробега от пикселя". Данная кодировка содержит два значения: первое -начальная позиция(номер пикселя); второе - длина пробега от этого пикселя. 
Пиксели нумеруются сверху вниз, затем слева направо: 1 - пиксель (1,1), 2 - пиксель (2,1) и т. д. 
Примеры:
Например значения «1 3» подразумевают начало пробега с пикселя (1,1)(включительно), длиною 3 пикселя, то есть пиксели(1,1); (1,2); (1,3).

Указание значений «1 3 10 5» подразумевает, что пиксели 1,2,3,10,11,12,13,14 включены в маску. 

Размер изображений тренировочной выборки 256(вертикаль) * 1600(горизонталь)

Для значения 29102 5:
стартовая позиция на пикселе: 
смещенному вправо по нашему изображению на 29102 // 256 = 113 пикселей
смещенному вниз на 29102 % 256 = 174 пикселя.
Получаем позицию с индексами (173, 113), учитываем, что индексация у нас начинается с 0.
Далее со стартовой позиции включаем в нашу маску ещё 4 пикселя вниз, итого 5 пикселей: 
(173, 112); (174, 112); (175, 112); (176, 112); (177, 112)

In [ ]:
def rle_decode_3(mask_rle:str, shape=(256, 1600))->NPArrayMask:
    "Return an image array from run-length encoded string `mask_rle` with `shape`."
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1    # Отнимаем от всех стартовых значений 1 т.к. индексация с 0.
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint)
    for a, b in zip(starts, ends): img[a:b] = 1
    img = img.reshape(shape, order='F')
    return img

In [ ]:
def open_mask_rle_2(mask_rle:str, shape=(256, 1600))->ImageSegment:
    "Return `ImageSegment` object create from run-length encoded string in `mask_lre` with size in `shape`."
    x = FloatTensor(rle_decode_3(str(mask_rle), shape).astype(np.uint8))
    x = x.view(-1, shape[0], shape[1])
    #return ImageSegment(x)
    return x

In [ ]:
def rle_encode_3(img:NPArrayMask, shape=(256, 1600))->str:
    "Return run-length encoding string from `img`."
    pixels = np.concatenate([[0], img.flatten(order='F') , [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
df_2['EncodedPixels'][1]

In [ ]:
rle_decode_3(df_2['EncodedPixels'][1][2])

In [ ]:
ImageSegment(open_mask_rle_2(df_2['EncodedPixels'][1][2]))

In [ ]:
mask = rle_decode_3(df_2['EncodedPixels'][1][2])

In [ ]:
type(mask)

In [ ]:
rle_encode_3(mask)

In [ ]:
fn = path_train + df_2['ImageId'][5]
img = open_image(fn)
img

In [ ]:
mask = open_mask_rle_2(df_2['EncodedPixels'].iloc[5][3])
ImageSegment(mask)

In [ ]:
mask = open_mask_rle_2(df_2['EncodedPixels'].iloc[5][2])
ImageSegment(mask)

In [ ]:
df_2[df_2['ImageId'] == 'db4867ee8.jpg']

In [ ]:
df_2['EncodedPixels'].iloc[40]

In [ ]:
df_2[df_2['ImageId'] == '000f6bf48.jpg']['EncodedPixels'].item()

In [ ]:
for i in df_2[df_2['ImageId'] == '000f6bf48.jpg']['EncodedPixels']:
    print(i)

In [ ]:
#fn = path_train + df_2.index['db4867ee8.jpg']
fn = path_train + '000f6bf48.jpg'
#print(fn)
img = open_image(fn)
shape = img.shape[-2:]
#print(shape)
#img = open_image(fn)
final_mask = torch.zeros((1, *shape))
#for i, rle in enumerate(df_2['EncodedPixels'].iloc[5]):
#for i, rle in enumerate(df_2['EncodedPixels'].loc['db4867ee8.jpg']):  
for i, rle in enumerate(df_2[df_2['ImageId'] == '000f6bf48.jpg']['EncodedPixels'].item()):
    #print(rle)
    if isinstance(rle, str):
        mask = open_mask_rle_2(rle)
        #print(mask.shape)
        final_mask += (i + 1) * mask
#mask = open_mask_rle_2(df_train['EncodedPixels'].iloc[0])
final_mask_2 = ImageSegment(final_mask)
_,axs = plt.subplots(3,1, figsize=(20,10))
img.show(ax=axs[0], title='no mask')
img.show(ax=axs[1], y=final_mask_2, title='masked')
final_mask_2.show(ax=axs[2], title='mask only', alpha=1.)

In [ ]:
def get_y_fn(fn):
    #print(fn)
    #fn = fn.replace(path_train, '')
    x = df_train[df_train['ImageId'] == fn]['EncodedPixels'].item()
    #print('!')
    return open_mask_rle_2(x)

In [ ]:
def get_y_fn_mcl(fn):
    #print(fn)
    final_mask = torch.zeros((1, 256, 1600))
    for i, rle in enumerate(df_2[df_2['ImageId'] == fn]['EncodedPixels'].item()):
    #for i, rle in enumerate(df_2['EncodedPixels'].loc[fn]):    
        if isinstance(rle, str):
            mask = open_mask_rle_2(rle)
            #print('mask=', mask.shape)
            final_mask += (i + 1) * mask
    return ImageSegment(final_mask)

In [ ]:
get_y_fn_mcl('000f6bf48.jpg')

In [ ]:
class SegmentationLabelList(SegmentationLabelList):
    def open(self, fn): return get_y_fn_mcl(fn)
    
class SegmentationItemList(SegmentationItemList):
    _label_cls = SegmentationLabelList

In [ ]:
path_train

In [ ]:
df_2.head()

In [ ]:
train_list = (SegmentationItemList
                .from_df(df_2, path_train))

In [ ]:
#codes = ['0','1']
train_list = (SegmentationItemList
                .from_df(df_2, path_train)
                .split_by_rand_pct()
                .label_from_df(cols='ImageId', label_cls=SegmentationLabelList, classes=[0, 1, 2, 3, 4])
                #.transform(get_transforms(), size=256,resize_method=ResizeMethod.SQUISH, tfm_y=True)
                .transform(get_transforms(), size=(128, 800),resize_method=ResizeMethod.SQUISH, tfm_y=True)
                #.transform(get_transforms(flip_vert=True), tfm_y=True)
                .databunch(bs=10, num_workers=10))

In [ ]:
type(train_list)

In [ ]:
train_list.show_batch(rows=3, figsize=(20,10))

In [ ]:
def dice_func(input, target):
    smooth = 0
    input = input[:,1,:,:]
    iflat = input.flatten().float()
    tflat = target.flatten().float()
    intersection = (iflat * tflat).sum()
    return ((2. * intersection + smooth) /
              (iflat.sum() + tflat.sum() + smooth))

def dice(input:Tensor, targs:Tensor, iou:bool=False)->Rank0Tensor:
    n = targs.shape[0]
    input = input.argmax(dim=1).view(n,-1)
    targs = targs.view(n,-1)
    intersect = (input * targs).sum().float()
    union = (input+targs).sum().float()
    if not iou: return (2. * intersect / union if union > 0 else union.new([1.]).squeeze())
    else: return intersect / (union-intersect+1.0)

In [ ]:
#learn = unet_learner(train_list, models.resnet18, pretrained=False, metrics=[dice], wd=1e-3, model_dir="/tmp/model/")

In [ ]:
learn = unet_learner(train_list, models.resnet18, pretrained=False, metrics=[dice], wd=1e-3, model_dir='/kaggle/working/models')

In [ ]:
lr_find(learn)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10,max_lr = 1e-2)    # 40

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(20,max_lr = slice(1e-3,1e-2))    # 40

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(20, max_lr = 1e-3)

In [ ]:
#learn.unfreeze()

In [ ]:
learn.fit_one_cycle(20, max_lr = 1e-4) 

In [ ]:
#learn.fit_one_cycle(20, max_lr = 1e-4)

In [ ]:
#learn.fit_one_cycle(20, max_lr = 1e-5)

In [ ]:
#learn.save('stage-1')
#learn.export("/kaggle/working/steel-1.pkl")

In [ ]:
learn.save('trained_model_1')
learn.export("/kaggle/working/trained_model_1.pkl")

In [ ]:
learn = load_learner('/kaggle/working/', 'trained_model_1.pkl')

In [ ]:
learn = unet_learner(train_list, models.resnet18, pretrained=False, metrics=[dice], wd=1e-3, model_dir='/kaggle/working/models')

In [ ]:
learn = learn.load("trained_model_1")

In [ ]:
learn.fit_one_cycle(10,max_lr = 1e-5)    # 40

In [ ]:
learn.recorder.plot_lr(show_moms=True)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.show_results()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
submit = pd.read_csv(path + 'sample_submission.csv', converters={'EncodedPixels': lambda e: ' '})
print(len(submit))

In [ ]:
submit.sort_values(by=['ImageId'])

In [ ]:
submit.head()

In [ ]:
learn.model.cuda()

In [ ]:
files = os.listdir(path=path_test)

In [ ]:
#a = learn.predict(open_image(path_test + '86c1f219f.jpg'))[1].data.numpy()
a = rle_decode_3(df_2['EncodedPixels'][1][2])
b = a.flatten(order='F')
d = {1: '', 2: '', 3: '', 4: ''}
for start, count in zip (rle_encode_3(a).split(' ')[::2], rle_encode_3(a).split(' ')[1::2]):
    #print(start)
    #print(b[int(start)-1])
    d[b[int(start)-1]] += str(start) + ' ' +  str(count) + ' '
d

In [ ]:
d[3] = '294661 251 294917 251 295173 251 295429 251 295685 251 295941 251 296197 251 296453 251 296709 251 296965 251 297221 251 297477 251'

In [ ]:
d

In [ ]:
img_name = '86c1f219f.jpg'

In [ ]:
sub_list = []
for i in d:
    sub_list += [[img_name, i, d[i]]]
sub_list

In [ ]:
sub_list = []
for img_name in files:
    #print(img_name)
    pred = learn.predict(open_image(path_test + img_name))[1].data.numpy()
    print(Imagese)
    #print(pred)
    pred_fl = pred.flatten(order='F')
    rle_enc = rle_encode_3(pred).split(' ')
    #print(rle_enc)
    d = {1: '', 2: '', 3: '', 4: ''}
    for start, count in zip (rle_enc[::2], rle_enc[1::2]):
        #print(start)
        #print(pred_fl[int(start)-1])
        d[pred_fl[int(start)-1]] += str(start) + ' ' +  str(count) + ' '
        
    for i in d:
        sub_list += [[img_name, i, d[i]]]


In [ ]:
df_2['EncodedPixels'].iloc[0]

In [ ]:
df_2['EncodedPixels'].iloc[0][0]

In [ ]:
pred = rle_decode_3(df_2['EncodedPixels'].iloc[0][0])

In [ ]:
pred_fl = pred.flatten(order='F')

In [ ]:
rle_enc = rle_encode_3(pred).split(' ')
rle_enc

In [ ]:
d = {1: '', 2: '', 3: '', 4: ''}
for start, count in zip (rle_enc[::2], rle_enc[1::2]):
    print(start)
    print(pred_fl[int(start)-1])
    d[pred_fl[int(start)-1]] += str(start) + ' ' +  str(count) + ' '

In [ ]:
d

In [ ]:
pred = learn.predict(open_image(path_test + '289d347d9.jpg'))[1].data.numpy()
pred_fl = pred.flatten(order='F')
rle_enc = rle_encode_3(pred).split(' ')


In [ ]:
rle_enc

In [ ]:
rle_enc

In [ ]:
pred_fl[55153+5]

In [ ]:
pred_fl[2740-1]

In [ ]:
pred_fl[2616-1]

In [ ]:
sub_list

In [ ]:
submission_df = pd.DataFrame(sub_list, columns=['ImageId', 'EncodedPixels', 'ClassId'])
submission_df.head()

In [ ]:
d

In [ ]:
b = np.where(a != 0)

In [ ]:
b

In [ ]:
a[0][38][612]

In [ ]:
test_count = len(files)
results = []

In [ ]:
def run_length(label_vec):
    encode_list = encode(label_vec)
    index = 1
    class_dict = {}
    for i in encode_list:
        if i[1] != len(codes)-1:
            if i[1] not in class_dict.keys():
                class_dict[i[1]] = []
            class_dict[i[1]] = class_dict[i[1]] + [index, i[0]]
        index += i[0]
    return class_dict

In [ ]:
from itertools import groupby

In [ ]:
def encode(input_string):
    return [(len(list(g)), k) for k,g in groupby(input_string)]

In [ ]:
codes = ['0','1','2','3', '4'] 

In [ ]:
learn.model.cuda()

In [ ]:
from tqdm import tqdm

In [ ]:
for i, img in tqdm(enumerate(files)):
    img_name = img
    pred = learn.predict(open_image(path_test + img))[1].data.numpy().flatten()
    class_dict = run_length(pred)
    print(class_dict)


In [ ]:
for i, img in tqdm(enumerate(files)):
    img_name = img
    pred = learn.predict(open_image(path_test + img))[1].data.numpy().flatten()
    class_dict = run_length(pred)
    if len(class_dict) == 0:
        for i in range(4):
            results.append([img_name+ "_" + str(i+1), ''])
    else:
        for key, val in class_dict.items():
            results.append([img_name + "_" + str(key+1), " ".join(map(str, val))])
        for i in range(4):
            if i not in class_dict.keys():
                results.append([img_name + "_" + str(i+1), ''])
        
        
    if i%20==0:
        print("\r{}/{}".format(i, test_count), end="")

In [ ]:
results

In [ ]:
files = list(path_test.glob("**/*.jpg"))

In [ ]:
def get_predictions(path_test, learn):
    # predicts = get_predictions(path_test, learn)
    learn.model.cuda()
    files = list(path_test.glob("**/*.jpg"))    #<---------- HERE
    test_count = len(files)
    results = []
    for i, img in enumerate(files):
        img_name = img.stem + '.jpg'
        pred = learn.predict(open_image(img))[1].data.numpy().flatten()
        class_dict = run_length(pred)
        if len(class_dict) == 0:
            for i in range(4):
                results.append([img_name+ "_" + str(i+1), ''])
        else:
            for key, val in class_dict.items():
                results.append([img_name + "_" + str(key+1), " ".join(map(str, val))])
            for i in range(4):
                if i not in class_dict.keys():
                    results.append([img_name + "_" + str(i+1), ''])
        
        
        if i%20==0:
            print("\r{}/{}".format(i, test_count), end="")
    return results    

sub_list = get_predictions(path_test, learn)

In [ ]:
test_list = (SegmentationItemList
                .from_df(df_2, path_train)
                .split_by_rand_pct()
                .label_from_df(cols='ImageId', label_cls=SegmentationLabelList, classes=[0, 1, 2, 3, 4])
                .transform(get_transforms(), size=256,resize_method=ResizeMethod.SQUISH, tfm_y=True)
                #.transform(get_transforms(flip_vert=True), tfm_y=True)
                .databunch(bs=10, num_workers=10))